In [1]:
import torch
import torch.nn as nn
import torchvision
from collections import OrderedDict
from torch.utils.data import DataLoader, Subset
import torch.nn.functional as F
import torchvision.transforms as transforms
import PIL
from tqdm import tqdm

from dataset import CelebA


class gender_classifier:

    def __init__(self):
        self.USE_CUDA = torch.cuda.is_available()
        self.DEVICE = torch.device('cuda' if self.USE_CUDA else 'cpu')
        model_ = torchvision.models.resnet101(
            weights=torchvision.models.ResNet101_Weights.DEFAULT)
        # model_ = torchvision.models.resnet101()
        n_inputs = model_.fc.in_features
        classifier = nn.Sequential(OrderedDict([
            ('fc1', nn.Linear(n_inputs, 1))
        ]))
        model_.fc = classifier
        # model_.load_state_dict(torch.load(
        # 'resnet_pretrain_b2.pt', map_location=self.DEVICE), strict=False)
        self.model = model_.to(self.DEVICE)
        self.transforms = transforms.Compose([transforms.ToTensor(),
                                              transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    def train(self, dataloader, model, criterion, optimizer, device):
        model.train()
        total_loss = 0
        for idx, (img, target) in tqdm(enumerate(dataloader), total = len(dataloader)):
            optimizer.zero_grad()
            img, target = img.to(device), target.to(device)
            output = F.sigmoid(model(img))
            loss = criterion(output, target)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
        return total_loss/len(dataloader)

    def train_model(self, epochs=100, learning_rate=0.001):
        train_data = CelebA()
        # used to reduce the size of the dataset from 200k to 10k
        indexes = [x for x in range(10000)]
        train_data_subset = Subset(train_data, indices = indexes)
        train_loader = DataLoader(train_data_subset, batch_size=32, shuffle=True)
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
        for epoch in range(epochs):
            epoch_loss = self.train(
                train_loader, self.model, criterion, optimizer, self.DEVICE)
            print(f'Epoch {epoch+1}: Loss {epoch_loss}', flush=True)
            torch.save(self.model.state_dict(), 'resnet_pretrain.pt')

    def classify_gender(self, image):
        self.model.eval()
        image = transforms.functional.rotate(img=image, angle=270)
        image = transforms.functional.center_crop(
            img=image, output_size=[448, 448])
        image = transforms.functional.resize(
            img=image, size=[224, 224]
        )
        to_return = image.copy()
        image = self.transforms(image).unsqueeze(0)
        output_ = self.model(image)
        output = F.sigmoid(output_)
        return output, to_return


In [2]:
instance = gender_classifier()
c_instance = CelebA()
print(c_instance.data[:5])

[('datasets/CelebA/Img/img_align_celeba/000001.jpg', (0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1)), ('datasets/CelebA/Img/img_align_celeba/000002.jpg', (0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1)), ('datasets/CelebA/Img/img_align_celeba/000003.jpg', (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1)), ('datasets/CelebA/Img/img_align_celeba/000004.jpg', (0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1)), ('datasets/CelebA/Img/img_align_celeba/000005.jpg', (0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1))]


In [10]:
import PIL
import pandas as pd
import os
def add_user_data (gender, image):
  user_dataset_location = "user_dataset"
  user_dataset_csv = os.path.join(user_dataset_location, "user_dataset_data.csv")
  template = [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1]
  gender = "male"
  if gender == "male":
    attr_ = 1
  else:
    attr_= 0
  template[20] = attr_
  new_image_attr = tuple(template)
  
  df = pd.read_csv(user_dataset_csv, index_col = False)
  last_image_name = df.loc[len(df)-1]['image_name'].split("_")
  image_number = int(last_image_name[-1].split(".")[0])
  new_image_name = f"user_image_{image_number+1}.jpg"
  new_image_path = os.path.join(user_dataset_location, new_image_name)
  image.save(new_image_path)
  df.loc[len(df)] = [new_image_path, new_image_attr]
  display(df)
  df.to_csv("./user_dataset/user_dataset_data.csv", index=False)


image = PIL.Image.open("000058.jpg")
add_user_data("female", image)



,image_name,new_image_attr
0,user_image_00.jpg,"(0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
1,user_dataset/user_image_1.jpg,"(0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."


In [11]:
instance = gender_classifier()
c_instance = CelebA()
print(c_instance.data[-1])

('datasets/CelebA/Img/img_align_celeba/202599.jpg', (0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1))


In [12]:
c_instance.add_user_dataset()
print(c_instance.data[-1])

('user_dataset/user_image_1.jpg', (0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1))


/home/UG/rng037/cz4042/dataset.py:58: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img_name = df.loc[row][0]
/home/UG/rng037/cz4042/dataset.py:61: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img_attr = literal_eval(df.loc[row][1])
